# Základní grafické operace – Numpy a Pillow

V těchto cvičeních budeme opět pracovat s obrázkem:

![Kvetina](images/kvetina/kvetina.jpg)

Byl vyfocen digitálním fotoaparátem a uložen v barvovém prostoru sRGB ve formátu JPEG. (A samozřejmě také výrazně zmenšen.) Jelikož úkolem tohoto cvičení je naučit se používat Numpy a další dostupné nástroje, pro načítání a ukládání už budeme používat metody knihovny Pillow.

Jsou-li k práci potřeba nějaké rovnice, jsou uvedeny v nápovědě (a také odkázány na zdroj), proto se jí nevyhýbejte.

In [ ]:
# Vynucení kontroly souladu s PEP8
!pip install flake8 pycodestyle pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
from PIL import Image
import numpy as np

## Načtení obrázku

In [ ]:
img = Image.open('images/kvetina/kvetina.jpg')
img

Pro převod `Image` objektu na `numpy` array lze použít `np.array` nebo `np.asarray`.

In [ ]:
data = np.asarray(img, dtype=np.float)
data.shape

Pro převod zpět na `Image` nejdříve pole přetypujte na pole celých čísel (8 bitové) a poté použijte [`Image.fromarray`](https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.fromarray).

In [ ]:
from IPython.display import display


def display_img(array, mode='RGB'):
    display(Image.fromarray(array.astype(np.int8), mode=mode))

## 1. Převeďte uvedený obrázek z pozitivu do negativu.

**Nápověda:**

Tohle je asi první cvičení na broadcasting – potřebujeme všechny barvové složky v každém pixelu odečíst od 255 a nemusíme kvůli tomu vyrábět matici 375×500×3.

In [ ]:
negativ = 255 - data
Image.fromarray(negativ.astype(np.int8), 'RGB')

## 2. Převeďte uvedený obrázek do odstínů šedi

**Nápověda:**

Pro převod použijte konverzní výraz podle ITU-R 601-2 luma transform pro převod z nelineárních hodnot R, G a B na tzv. lumu (nikoli tedy luminanci!):

$$Y_{601} = 0.299 * R + 0.587 * G + 0.114 * B$$

PS: Pro podrobnosti viz [What is "luma"?](http://www.poynton.com/notes/colour_and_gamma/ColorFAQ.html#RTFToC11).

In [ ]:
# gray = data.mean(axis=2)
gray = np.average(data, weights=[0.299, 0.587, 0.114], axis=2)
# gray = (data * [0.299, 0.587, 0.114]).sum(axis=2)

Image.fromarray(gray.astype(np.int8), 'L')

## 3. Nyní v rámci procvičení výřezů numpyovských polí vyextrahujte z originálního obrázku jednotlivé barevné kanály

**Nápověda:**

Jedna barvová složka představuje světlost v příslušném jednom barevném kanálu, proto se zobrazují v odstínech šedi. Čistě teoreticky si můžete dát tu práci a pokusit se je zobrazit v „přirozené“ barvě, ale jelikož to na webu umíme jenom přes plné RGB-spektrum, není to vůbec jednoduché.

In [ ]:
from IPython.display import display

for i in range(3):
    out = data[:, :, i]
    display(Image.fromarray(out.astype(np.int8), 'L'))

## 4. Jako další procvičení na výřezy numpyovských polí zmenšete vstupní obrázek třebas na polovinu

**Nápověda:**

Při zmenšování jde o ztrátu informace a překvapivě se dá rozumných (tedy koukatelných) výsledků dosáhnout pouhým výběrem podmnožiny všech pixelů, což je smyslem této úlohy. Správně by se „zahazované“ pixely nějakým způsobem průměrovaly, ale to si necháme na jindy.

In [ ]:
out = data[::2, ::2, :]
Image.fromarray(out.astype(np.int8), 'RGB')

## 5. Jako další operaci na broadcasting zkuste originální obrázek ztmavit

**Nápověda:**

Ztmavit obrázek znamená zmenšit světlost v jeho jednotlivých barvových kanálech. V nejjednodušším případě ve všech stejně, jedná se tedy o broadcasting skaláru. Reálně by se spíše počítal každý kanál jiným koeficientem a šlo by tak o broadcasting vektorem, ale to si také necháme na jindy. (Natožpak pak ztmavování pomocí vhodné nelineární funkce.)

Mimochodem – při ztmavování obrázku pochopitelně začneme přicházet o informace, především ve stínech. Za prvé menší rozdíly asi nerozeznáme tak dobře a za druhé zmenšováním čísel blízko nuly můžeme informaci prostě ztratit i díky konečné přesnosti výpočtu. V praxi se snadno a rychle stane, že se tmavé plochy začnou za chvíli všechny mapovat na jednolitou černou.

In [ ]:
out = data * 0.5
Image.fromarray(out.astype(np.int8), 'RGB')

## 6. Doplňkovou operací ke ztmavení je zesvětlání

Pozor – tato úloha už není tak přímočará jako předchozí na ztmavení!

**Nápověda:**

Zcela naopak k předchozí úloze musíme nyní světlost v jednotlivých barvových kanálech zvětšit (a zase v zájmu zjednodušení použijeme pouze broadcasting skalárem místo vektorem nebo dokonce rovnou nějakou nelineární funkcí). Jenomže zatímco přibližovat se k nule jsme mohli bez nesnází, jen tak si zvětšovat čísla dost dobře nemůžeme – typický výstupní obrázek má na jeden barvový kanál vyhrazen pouze jeden bajt a do něj se vleze nejvyšší číslo 255. 

Je jasné, že vypočítané hodnoty musíme držet v rozmezí 0-255 a že tak při zesvětlování začneme ztrácet informace nejdříve z těch světlejších míst – prostě se dřív všechny namapují na čistou bílou. (A když to přeženete, tak tam kromě čisté černé namapujete úplně všechno.)

In [ ]:
out = np.clip(data * 2, 0, 255)
Image.fromarray(out.astype(np.int8), 'RGB')

## 7. Ať si vyzkoušíme také něco složitějšího než jen násobení skalárem – „vypněte“ ve výstupních obrázcích vždy jeden kanál a druhé dva zachovejte

In [ ]:
# out = data * np.array([0, 1, 1])
out = data * np.array([1, 0, 1])
# out = data * np.array([1, 1, 0])

Image.fromarray(out.astype(np.int8), 'RGB')

## 8. Na závěr si zahrajeme na analogového fotografa vybaveného digitální technikou

Dostali jste barevný negativ a máte za úkol ho zkopírovat (tedy zachovat velikost) pomocí zvětšovacího přístroje se zeleným filtrem. Jelikož ale nemáte možnost změřit osvit pozitivu, musíte provést klasickou proužkovou zkoušku – rozdělit pozitiv na několik (obdélníkových) oblastí a postupným posouváním krycí masky po nich vyzkoušet, jak se délka osvitu projeví (samozřejmě pouze v dané části obrazu). Programově tedy:

* Výstupem je pozitiv, musíte proto barvy převést z negativu.
* Zelený filtr je ideální, ze tří barvových složek originálu tak použijete pouze zelený kanál.
* „Osvit“ testujte v pěti oblastech následujícím způsobem – prostřední část prostě zkopírujte, dvě krajní na jedné straně „podsviťte“ (tedy ztmavte) a druhé dvě krajní naopak „přesviťte“ (tedy zesvětlete).

V zájmu procvičení práce s numpy-poli spočítejte výstup pro horizontální i vertikální směr.

**Nápověda:**

Možná trochu zamotané, ale relativně přímočaré cvičení. Jen si dejte pozor, abyste nepřišli o originální obrázek – potřebujete ho pro dva výpočty.

In [ ]:
from IPython.display import display

obr = 255 - data[:, :, 1]
obr_copy = np.copy(obr)

osvit = 0.25, 0.5, 1, 2, 4

for i, j in enumerate(range(0, 500, 100)):
    pohled = obr[:, i*100:i*100+100]
    pohled *= osvit[i]

obr = np.clip(obr, 0, 255)
display(Image.fromarray(obr.astype(dtype=np.uint8), 'L'))

osvit = 0.3, 0.6, 1, 1.3, 1.6

for i, j in enumerate(range(0, 375, 75)):
    pohled = obr_copy[i*75:i*75+75, :]
    pohled *= osvit[i]

obr_copy = np.clip(obr_copy, 0, 255)
display(Image.fromarray(obr_copy.astype(dtype=np.uint8), 'L'))